<a href="https://colab.research.google.com/github/compartia/AI-tecture/blob/master/miro_openai_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare OPEN AI client

In [ ]:
!pip install --upgrade openai typing-extensions

In [ ]:
import openai
from google.colab import userdata
# openai.api_key =

ai_client = openai.OpenAI(api_key=userdata.get('OP_API')  )

In [ ]:
BOARD_ID = 'uXjVN61x8Pg='

# Miro client

In [ ]:
import requests
import pandas as pd


class MiroClient:
  def __init__(self, token, board_id):
    self.board_id = board_id
    self.token = token
    self.base_url = f"https://api.miro.com/v2/boards/{board_id}"

    self.headers = {
        "accept": "application/json",
        "authorization": f"Bearer {self.token}"
    }


  def get_items(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/items?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/items?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response

  def get_connectors(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/connectors?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/connectors?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response


  def get_all_pages(self, page_method):
    # print('get_all_pages, page_method=', page_method)
    cursor = None

    while True:
      response = page_method(cursor)
      rj = response.json()
      yield rj
      cursor = rj.get('cursor')
      if cursor is None:
        #TODO: safety! add limit : we trust 3rd pary apy too much, what if cursor is never None?
        break


miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)


In [ ]:
r = miro_client.get_items(limit=50).json()

for k in r['data']:
  print()
  print('-'*30)
  print(k)

In [ ]:
type(r['data'])

## Getting all connectors

In [ ]:
def decode_connectors(page:dict):
  # print(page)
  for i in page['data']:
    yield i['id'], i.get('startItem', {}).get('id'), i.get('endItem', {}).get('id')

## test it
one_page_con = miro_client.get_connectors(limit=10).json()
x = [i for i in decode_connectors(one_page_con)]
x

In [ ]:
def get_all_connectors(self):
  for page in self.get_all_pages(self.get_connectors):
    # print('get_all_connectors, page:', page)
    for i in decode_connectors(page):
      yield i


MiroClient.get_all_connectors = get_all_connectors


##-----
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
all_connectors_iter = miro_client.get_all_connectors()

connectors_df = pd.DataFrame(all_connectors_iter, columns=['id','id_from', 'id_to'])
connectors_df = connectors_df.set_index('id')
connectors_df


## Getting all shapes (widgets)

In [ ]:
def decode_items(page:dict):

  for i in page['data']:
    # print(type(i))
    if type(i) == dict :
      yield i['id'], i['type'], i['data'].get('shape', '_undefined_'), i['data'].get('content'), i['modifiedAt'], i['geometry'], i['position']

    else:
      print('NOT A DICT!!', type(i), i)
      decode_items(i)

In [ ]:


def get_all_items(self):
  for page in self.get_all_pages(self.get_items):
    # print( page.get('cursor'), page.get('size'), page.get('total') )
    for i in decode_items(page):
      yield i
      # print(i)


MiroClient.get_all_items=get_all_items

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
items_iter = miro_client.get_all_items()

df = pd.DataFrame(items_iter, columns=['id','type', 'shape', "contents", 'modifiedAt', 'geometry', 'position'])
df = df.set_index('id')

In [ ]:
shapes_df =df
shapes_df

In [ ]:
# shapes_df.at['3458764578193755154', 'type']

### Get shapes with prompts

In [ ]:
prompts_df = shapes_df[shapes_df['shape']=='parallelogram']
prompts_df

# Test open AI

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
%%time

completion = ai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

# print(completion.choices[0].message)
display(HTML(completion.choices[0].message.content))

In [ ]:
%%time
completion = ai_client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
#
# print(completion.choices[0].message)
display(HTML(completion.choices[0].message.content))

In [ ]:
%%time
completion = ai_client.chat.completions.create(
  model="gpt-4-turbo-preview",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

display(HTML(completion.choices[0].message.content))

In [ ]:
def ask_ai(prompt, argument_context):

  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": f"{prompt}. Your answer is in a form of list of <p> html tags and NO newline '\n' symbols."},
      {"role": "user", "content": argument_context}
    ]
  )
  return completion.choices[0].message.content

r = ask_ai('you are the philosopher. answer in 50 words.', 'what is the meaning of life?')
print(r)

In [ ]:
display(HTML(r))
display(HTML(r))

# Process

### Update board item

In [ ]:
# shapes_df.loc['3458764578193755154' ]

In [ ]:


def get_shape_info(self, shape_id):
  url = f"{self.base_url}/shapes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)

def get_stiky_note_info(self, shape_id):
  url = f"{self.base_url}/sticky_notes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)




def update_item(self, shape_id, item_type, content, shape=None):


  url = None

  payload = { "data": {
          "content": content
      } }



  if item_type=='sticky_note':
    url = f"{self.base_url}/sticky_notes/{shape_id}"
  elif item_type=='shape':
    url = f"{self.base_url}/shapes/{shape_id}"
    if shape is not None:
      payload['data']['shape'] = shape




  response = requests.patch(url, json=payload, headers=self.headers)

  print(response.text)

MiroClient.update_item = update_item
MiroClient.get_shape_info = get_shape_info
MiroClient.get_stiky_note_info = get_stiky_note_info

#-----


miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
# miro_client.get_stiky_note_info('3458764578213963472')
miro_client.update_item( '3458764578213963472', item_type=shapes_df.at['3458764578213963472', 'type'], content=completion.choices[0].message.content,  shape='round_rectangle')

### Find connectors of prompts-shapes (connectors of prallelograms)

In [ ]:
#incoming shapes

ai_responses={}

for prompt_id in prompts_df.index:
  print()
  print('-'*40)
  # print('prompt', id, prompts_df.loc[id]['contents'])

  _propmt_text = shapes_df.loc[prompt_id]['contents']

  incoming_links = connectors_df[ connectors_df['id_to']==prompt_id]
  # print (links ['id_from'])
  for _, l in incoming_links.iterrows():
    print(l)
    print()
    incoming_shape_id = l['id_from']
    print('SHAPE  FROM:', incoming_shape_id)
    if incoming_shape_id in shapes_df.index:

      _text = shapes_df.loc[incoming_shape_id]['contents']
      print('text:', prompt_id, _text)
      r = ask_ai(_propmt_text, _text)
      display(HTML(r))
      ai_responses[prompt_id] = r

    else:
      print('ERROR!', incoming_shape_id, 'not found!' )




  print('='*80)
  print('outputs:')

  outgoing_links = connectors_df[ connectors_df['id_from'] ==prompt_id]
  # print (links ['id_from'])
  for _, l in outgoing_links.iterrows():
    print(l)
    print()
    out_shape_id = l['id_to']
    print('SHAPE to:', out_shape_id)

    if out_shape_id in shapes_df.index:
      print('text:', prompt_id, shapes_df.loc[out_shape_id]['contents'])
      # miro_client.update_item(out_shape_id, , content=r, shape='rou')

      miro_client.update_item( out_shape_id, item_type=shapes_df.at[out_shape_id, 'type'], content=r,  shape='round_rectangle')

    else:
      print('ERROR!', out_shape_id, 'not found!' )

  #   print(l)